# Transformer

### 1. 简介
Transformer 架构是2017年提出的一种基于自注意力机制的深度学习模型，它彻底改变了自然语言处理领域，并成为现代大语言模型的核心基础。

论文链接: https://arxiv.org/abs/1706.03762

### 2. 模型架构

#### 位置编码 positional encoding
Transformer完全基于注意力机制实现语义捕捉，没有使用到循环神经网络，因此无法捕捉序列顺序（位置）信息，
位置编码是对序列的位置进行编码，使位置成为一个向量，这个向量与序列的内容无关，只和序列长度相关。
其公式如下：

$ PE(pos, 2i) = sin(\frac {pos}{10000 ^ {\frac {2i}{d}}}) $

$ PE(pos, 2i+1) = cos(\frac {pos}{10000 ^ {\frac {2i}{d}}}) $

公式解释：假设我们有五个单词组成的序列，我们使用d=4的嵌入维度的向量来表示一个单词，
那么假设第一个单词的嵌入向量是[0.011, 0.025, 0.023, 0.256]，长度为4，
在Transofomer架构中，为了让位置向量和嵌入向量能够相加，位置向量和嵌入向量的长度相同，
因此位置向量的长度应该为4，由于是第一个单词，在序列的位置中为0，所以pos=0，
即pos表示序列的绝对位置，2i和2i+1的作用仅仅是为了分出奇偶，

我们计算当i=0时的结果：2i=0, 2i+1=1, 即PE(pos=0, 2i=0) = 0，PE(pos=0, 2i=1) = 1，
即此时有两个值：0和1，

我们再来计算当i=1时的结果：2i=2, 2i+1=3，即PE(pos=0, 2i=2) = 0，
PE(pos=0, 2i+1=3) = 1，即此时有四个值：0，1，0，1。

此时值的数量已经足够，不必再往下算，因此位置向量为[0, 1, 0, 1]，
相应的，当我们计算第二个单词的位置向量时，其pos=1，有位置向量[0.8415, 0.5403, 0.0001, 1.0000]，
这两个向量可以均可以与所在位置单词的嵌入向量相加。
得到的结果便同时存在单词本身信息与位置信息。

#### 注意力机制
注意力机制通过计算查询向量和一组键值对的相关性来工作。根据Q、K、V的来源不同，主要分为两种：

自注意力（self-attention）：Q、K、V全部来源于同一个序列。例如，Transformer的Encoder中，每个词都会关注输入句子中的所有词，以充分理解上下文；在Decoder中，也会有一个掩码自注意力层，让每个词关注它之前的所有词。

掩码自注意力（masked self-attention）：与自注意力相同，但是会对未来位置进行掩盖，以防止信息泄露。
同时它还让transformer具备了并行计算的能力，如果像RNN一样逐词训练，则不具备该能力。

交叉注意力：Q来源于一个序列，而K、V来源于另一个序列。这主要用于Encoder-Decoder架构，典型例子是Transformer的Decoder层，在生成目标词时，其Q会与来自Encoder输出的键值对K, V进行计算，从而聚焦于源序列中最相关的部分来进行翻译或生成。

多头注意力：多个自注意力的结合。


#### 层归一化 Layer Normalization
不同于批归一化（Batch Normalization）是对整个批次的每个特征进行归一化，

层归一化是对每一行进行归一操作，即对每一个样本进行归一化操作，更加适用于RNN和Transformer。

例如有数据如下所示：

[
[1, 2, 3, 4],
[5, 6, 7, 8]
]

层归一化对[1, 2, 3, 4]进行归一化，批归一化对[1, 5]进行归一化。

### 3. 代码实现
代码来源于哈佛NLP团队：https://github.com/harvardnlp/annotated-transformer

代码较难理解，可结合网络各种资源和AI进行学习。

In [ ]:
import torch
import math
import copy
import torch.nn as nn

from torch.nn.functional import log_softmax

In [ ]:
class EncoderDecoder(nn.Module):
    "标准的编码器-解码器架构。"
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    '''
    标准的全连接层+softmax层生成器。
    是编码器-解码器架构的输出层，
    用于将解码器的输出转换为词汇表上的概率分布。
    '''

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [ ]:
class LayerNorm(nn.Module):
    "层归一化。"

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

'''
Pytorch中有内置函数如下：
import torch.nn as nn
builtin_ln = nn.LayerNorm(normalized_shape=shape, eps=eps)
'''

In [ ]:
def clones(module, N):
    "克隆N个相同层。"
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [ ]:
class SublayerConnection(nn.Module):
    "子连接层。残差连接+归一化，即Add & Norm层"
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderLayer(nn.Module):
    "编码器层"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [ ]:
class Decoder(nn.Module):
    "解码器"
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):
    "解码器层"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn # 带掩码的自注意力
        self.src_attn = src_attn # 编码器-解码器注意力
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [ ]:
def subsequent_mask(size):
    '''
    创建一个下三角矩阵
    用于确保解码器在预测当前位置时只能看到当前位置及之前的位置，
    不能看到未来的位置。
    '''
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    "注意力机制"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "多头注意力机制"
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x)

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "位置前馈网络"
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
    "位置编码层"
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe) # 注册为buffer，不参与训练

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [ ]:
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1
):
    c = copy.deepcopy # 深拷贝，确保每个层有独立的参数实例
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab), # 线性层+softmax
    )

    # Xavier均匀初始化
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model